## Dehli-Indian-Housing-Analysis-Part-4-Predict-Price-with-Location-Name-compare-with-Prediciting-with-House-Type by Dumisani Maxwell Mukuchura

#### Dataset Source: *https://www.kaggle.com/datasets/bhavyadhingra00020/india-rental-house-price*

##### Contact: dumisanimukuchura@gmail.com | LinkedIn: https://www.linkedin.com/in/dumisani-maxwell-mukuchura-4859b7170/

##### This Project will explore the Dehli Indian Housing and build a model predicting rental price from Location Name and also from House Type i.e Compare of the 2 features which is better at Predicting Price.

Building from Part 3 of this series we noticed Location in terms of Latitude and Longidude coordinates it was not the best Feature to use to Predict the Rental Prices.

## 1. Prepare Data

### Modules and Library Import

In [7]:
# Library and Module import

import os #For File and Directory Operations

#For Data Manipulation and Visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#For One Hot Encoding Categorical Features
from category_encoders import OneHotEncoder

#For splitting the Data into Training and Test Data
from sklearn.model_selection import train_test_split

#For Model Building
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.utils.validation import check_is_fitted

In [2]:
# Import Data building from Part 3 where we have a Clean Data Frame

#Get the current working directory
current_dir = os.getcwd()

#Trim to one level above
base_dir = os.path.dirname(current_dir)

#Construct the path to get the Data Folder
data_dir = os.path.join(base_dir, "data")

#Construct the full path to the CSV file
csv_file_path = os.path.join(data_dir, "Dehli-Indian-Housing-Clean-Data.csv")

#Read the csv file into a DataFrame
housing_data = pd.read_csv(csv_file_path)

#Make a copy to keep the original dataset intact
housing_df = housing_data.copy()

housing_df.head()

,house_type,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,verificationDate,description,SecurityDeposit,Status,numBedroom,house_layout,house_size_in_sqft,price_approx_usd
0,Independent Floor,Guru Angad Nagar,Delhi,28.635309,77.224960,13000,INR,2.0,1.0,Negotiable,0.29,Posted 4 years ago,Itâs a 2 bhk builder floor situated in Guru ...,"13,000",Unfurnished,2,BHK,517.0,151.59
1,Independent House,Tuglak Road,Delhi,28.595453,77.210335,2162955,INR,4.0,2.0,Not Negotiable,4.03,Posted a year ago,Well designed 5 bhk independent house is avail...,"65,41,919",Unfurnished,5,BHK,6251.0,25221.02
2,Villa,Tuglak Road,Delhi,28.595453,77.210335,2010105,INR,4.0,2.0,Not Negotiable,4.21,Posted a year ago,Its four bhk villa in the super location of de...,"60,10,155",Unfurnished,4,BHK,5562.0,23438.72
3,Independent Floor,Saket,Delhi,28.523548,77.200142,235562,INR,3.0,2.0,Not Negotiable,0.96,Posted a year ago,Itâs a 3 bhk builder floor situated in Saket...,"7,51,225",Unfurnished,3,BHK,2855.0,2746.76
4,Independent Floor,Saket,Delhi,28.523548,77.200142,301011,INR,3.0,2.0,Not Negotiable,1.28,Posted a year ago,Well designed 3 bhk builder floor is available...,"10,10,202",Unfurnished,3,BHK,2741.0,3509.92


## 2. Explore  

### Understand the Qualities of the 2 Features we shall need 'house_type' , 'location'

In [3]:
# Find how many unique values are within the 2 Features we shall need to explore 

print("Unique Values within Location Name:", housing_df["location"].nunique())

Unique Values within Location Name: 139


In [5]:
print("Unique Values within Location Name:", housing_df["house_type"].nunique())

Unique Values within Location Name: 6


In [7]:
#Print the Unique Values in Location Name column

print(housing_df["location"].unique())

['Guru Angad Nagar' 'Tuglak Road' 'Saket' 'Maharani Bagh'
 'New Friends Colony' 'Anand Niketan' 'Golf Links' 'Friends Colony'
 'Geetanjali Enclave' 'Sarvodaya Enclave' 'New Moti Nagar' 'Shivalik'
 'Malviya Nagar' 'Shahpur Jat Village' 'Sheikh Sarai' 'Naraina Vihar'
 'Karampura' 'laxmi nagar' 'Mayur Vihar II' 'Rajinder Nagar'
 'Vasant Vihar' 'Janakpuri' 'Greater Kailash' 'Defence Colony'
 'Sunder Nagar' 'Jor bagh' 'mayur vihar phase 1' 'Shanti Niketan'
 'Mayur Vihar Phase II' 'Niti Bagh' 'Safdarjung Enclave' 'Sarvpriya Vihar'
 'West End' 'Sainik Farm' 'Hauz Khas' 'Navjeevan Vihar' 'Ghitorni'
 'Vasant Kunj' 'Chattarpur' 'Prithviraj Road' 'Malcha Marg'
 'Greater Kailash II' 'Panchsheel Park' 'Panchsheel Enclave' 'Lodhi Road'
 'Tilak Marg' 'Gulmohar park' 'Mahavir Enclave' 'Sector 2 Dwarka'
 'Sector 3 Dwarka' 'New Rajinder Nagar' 'Aurungzeb Road'
 'Amrita Shergill Marg' 'Babar Road' 'Lodhi Gardens' 'Lodhi Estate'
 'Green Park Extension' 'Green Park' 'Uday Park' 'East Patel Nagar'
 'New Raj

In [6]:
#Print the Unique Values in House Type column 

print(housing_df["house_type"].unique())

['Independent Floor ' 'Independent House ' 'Villa ' 'Apartment '
 'Studio Apartment ' 'penthouse ']


From this Exploration it is clear that the Location Names are many; 139 of them and we must navigate such High Cardinal Feature in a way that keeps perfomance without eating away compute, on the other end with House Type we only have 6 which can be worked with/ We proceed now to deal with the categorical Features for the Model Building

In [9]:
# Check the number of Non Null Values in the Features of Interest

housing_df[["house_type", "location"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2171 entries, 0 to 2170
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   house_type  2171 non-null   object
 1   location    2171 non-null   object
dtypes: object(2)
memory usage: 34.1+ KB


There are no Null Values within those Features.

## 3. Split Data 

### Splitting Data for Location Feature

In [10]:
# Splitting Data for Location Feature
feature = ["location"]
target = "price_approx_usd"
X_loc = housing_df[feature]
y_loc = housing_df[target] 

X_train_loc, X_test_loc, y_train_loc, y_test_loc = train_test_split(X_loc, y_loc, test_size=0.2, random_state=42)  # test_size=0.2 means 20% of data goes to test set

#Confirm the Split 
print("X_train_loc:\n", X_train_loc.head())
print("X_test_loc:\n", X_test_loc.head())
print("y_train_loc:\n", y_train_loc.head())
print("y_test_loc:\n", y_test_loc.head())

X_train_loc:
                 location
555                Saket
1435        Vasant Vihar
588        Gulmohar park
1532  Safdarjung Enclave
20         Anand Niketan
X_test_loc:
              location
1652         Jor bagh
342      Sunder Nagar
1342  Panchsheel Park
765     Anand Niketan
865      Vasant Vihar
y_train_loc:
 555     3510.05
1435    3512.41
588     3511.12
1532    7463.90
20      4222.55
Name: price_approx_usd, dtype: float64
y_test_loc:
 1652    5877.04
342     3509.93
1342     757.93
765     3509.93
865     3509.92
Name: price_approx_usd, dtype: float64


### Splitting Data for House Type Feature

In [11]:
# Splitting Data for Location Feature
feature = ["house_type"]
target = "price_approx_usd"
X_hse_type = housing_df[feature]
y_hse_type = housing_df[target] 

X_train_hse_type, X_test_hse_type, y_train_hse_type, y_test_hse_type = train_test_split(X_hse_type, y_hse_type, test_size=0.2, random_state=42)  # test_size=0.2 means 20% of data goes to test set

#Confirm the Split 
print("X_train_hse_type:\n", X_train_hse_type.head())
print("X_test_hse_type:\n", X_test_hse_type.head())
print("y_train_hse_type:\n", y_train_hse_type.head())
print("y_test_hse_type:\n", y_test_hse_type.head())

X_train_hse_type:
               house_type
555   Independent Floor 
1435  Independent Floor 
588   Independent Floor 
1532  Independent House 
20    Independent Floor 
X_test_hse_type:
               house_type
1652  Independent Floor 
342   Independent Floor 
1342  Independent Floor 
765   Independent Floor 
865   Independent Floor 
y_train_hse_type:
 555     3510.05
1435    3512.41
588     3511.12
1532    7463.90
20      4222.55
Name: price_approx_usd, dtype: float64
y_test_hse_type:
 1652    5877.04
342     3509.93
1342     757.93
765     3509.93
865     3509.92
Name: price_approx_usd, dtype: float64
